This is the implementation of ATBD approach 2.1. Users need to read and run `Transient_base.ipynb` before reading and runing this notebook.

In approach 2.1, the number of pixel pairs which meet the mission requirement as a percentage of the total number of pixel pairs selected are counted.

In [1]:
import numpy as np
import pickle
from pathlib import Path
import pandas as pd

In [2]:
calval_dir = Path.cwd()/'calval'
calval_location = 'central_valley'
# calval_location = 'texas'
# calval_location = 'oklahoma'
# calval_location = 'purtorico'
work_dir = calval_dir/calval_location

In [3]:
with open(work_dir/'approach2.pkl','rb') as f:
    dist,rel_measure, ifgs_date = pickle.load(f)

In [4]:
n_ifgs = len(dist)

In [5]:
n_bins = 10
bins = np.linspace(0.1,50.0,num=n_bins+1)

In [6]:
n_all = np.empty([n_ifgs,n_bins+1],dtype=int) # number of points for each ifgs and bins
n_pass = np.empty([n_ifgs,n_bins+1],dtype=int) # number of points pass
#ratio = np.empty([n_ifgs,n_bins+1]) # ratio
# the final column is the ratio as a whole
for i in range(n_ifgs):
    inds = np.digitize(dist[i],bins)
    for j in range(1,n_bins+1):
        rqmt = 3*(1+np.sqrt(dist[i][inds==j]))# mission requirement for i-th ifgs and j-th bins
        rem = rel_measure[i][inds==j] # relative measurement
        assert len(rqmt) == len(rem)
        n_all[i,j-1] = len(rem)
        n_pass[i,j-1] = np.count_nonzero(rem<rqmt)
    n_all[i,-1] = np.sum(n_all[i,0:-2])
    n_pass[i,-1] = np.sum(n_pass[i,0:-2])

In [7]:
def to_str(x:bool):
    if x==True:
        return 'true '
    elif x==False:
        return 'false '

In [8]:
ratio = n_pass/n_all
mean_ratio = np.array([np.mean(ratio[:,:-1],axis=1)])
ratio = np.hstack((ratio,mean_ratio.T))
thresthod = 0.683
#The assumed nature of Gaussian distribution gives a probability of 0.683 of being within one standard deviation.
success_or_fail = ratio>thresthod
success_or_fail_str = [list(map(to_str, x)) for x in success_or_fail]

In [9]:
columns = []
for i in range(n_bins):
    columns.append(f'{bins[i]:.2f}-{bins[i+1]:.2f}')
columns.append('total')

In [10]:
index = []
for i in range(len(ifgs_date)):
    index.append(ifgs_date[i,0].strftime('%Y%m%d')+'-'+ifgs_date[i,1].strftime('%Y%m%d'))

In [11]:
n_all_pd = pd.DataFrame(n_all,columns=columns,index=index)
n_pass_pd = pd.DataFrame(n_pass,columns=columns,index=index)
ratio_pd = pd.DataFrame(ratio,columns=columns+['mean'],index=index)
success_or_fail_pd = pd.DataFrame(success_or_fail_str,columns=columns+['mean'],index=index)

Number of data points in each bin:

In [12]:
n_all_pd

,0.10-5.09,5.09-10.08,10.08-15.07,15.07-20.06,20.06-25.05,25.05-30.04,30.04-35.03,35.03-40.02,40.02-45.01,45.01-50.00,total
20170108-20170120,1682,4603,7240,9578,11818,13545,15159,16554,17725,18500,97904
20170114-20170126,1625,4819,7275,9828,11760,13676,15135,16558,17466,18359,98142
20170120-20170201,1691,4622,7230,9738,11755,13647,15099,16419,17698,18298,97899
20170225-20170309,1537,4721,7234,9606,11876,13666,15151,16539,17539,18498,97869
20170321-20170402,1648,4683,7236,9613,11844,13773,15210,16549,17639,18462,98195
20170414-20170426,1673,4661,7181,9643,12043,13675,15260,16712,17422,18537,98270
20170508-20170520,1601,4497,7337,9789,11803,13741,15186,16371,17765,18501,98090
20170601-20170613,1627,4613,7282,9694,11640,13774,15336,16489,17469,18456,97924
20170625-20170707,1666,4556,7342,9780,11872,13774,15260,16761,17686,18393,98697
20170719-20170731,1600,4601,7361,9705,11711,13521,15219,16344,17662,18316,97724


Number of data points that below the curve:

In [13]:
n_pass_pd

,0.10-5.09,5.09-10.08,10.08-15.07,15.07-20.06,20.06-25.05,25.05-30.04,30.04-35.03,35.03-40.02,40.02-45.01,45.01-50.00,total
20170108-20170120,1374,3735,5997,8158,10089,11728,13264,14493,15674,16345,84512
20170114-20170126,1427,4221,6441,8844,10621,12516,13826,15211,15997,16962,89104
20170120-20170201,1418,3754,5919,8085,9744,11322,12580,13566,14666,15161,81054
20170225-20170309,1411,4321,6578,8714,10653,12135,13384,14450,15320,16128,86966
20170321-20170402,1294,3212,4733,5879,6810,7591,8126,8738,8993,9445,55376
20170414-20170426,1599,4373,6591,8698,10430,11349,11957,12254,11810,11794,79061
20170508-20170520,1482,4178,6756,9073,10922,12761,14133,15208,16484,17160,90997
20170601-20170613,1508,4201,6571,8801,10501,12326,13671,14605,15358,16156,87542
20170625-20170707,1517,4012,6171,7925,9421,10592,11490,12478,13098,13794,76704
20170719-20170731,1517,4141,6373,8028,9400,10599,11562,12214,12848,13076,76682


Ratio of pass:

In [14]:
s = ratio_pd.style
s.set_table_styles([  # create internal CSS classes
    {'selector': '.true', 'props': 'background-color: #e6ffe6;'},
    {'selector': '.false', 'props': 'background-color: #ffe6e6;'},
], overwrite=False)
s.set_td_classes(success_or_fail_pd)

,0.10-5.09,5.09-10.08,10.08-15.07,15.07-20.06,20.06-25.05,25.05-30.04,30.04-35.03,35.03-40.02,40.02-45.01,45.01-50.00,total,mean
20170108-20170120,0.816885,0.811427,0.828315,0.851744,0.853698,0.865855,0.874992,0.875498,0.884288,0.883514,0.863213,0.854621
20170114-20170126,0.878154,0.875908,0.885361,0.899878,0.903146,0.915180,0.913512,0.918650,0.915894,0.923907,0.907909,0.902959
20170120-20170201,0.838557,0.812203,0.818672,0.830253,0.828924,0.829633,0.833168,0.826238,0.828681,0.828560,0.827935,0.827489
20170225-20170309,0.918022,0.915272,0.909317,0.907141,0.897019,0.887970,0.883374,0.873692,0.873482,0.871878,0.888596,0.893717
20170321-20170402,0.785194,0.685885,0.654091,0.611568,0.574975,0.551151,0.534254,0.528008,0.509836,0.511591,0.563939,0.594655
20170414-20170426,0.955768,0.938211,0.917839,0.902001,0.866063,0.829909,0.783552,0.733246,0.677879,0.636241,0.804528,0.824071
20170508-20170520,0.925671,0.929064,0.920812,0.926857,0.925358,0.928681,0.930660,0.928960,0.927892,0.927517,0.927689,0.927147
20170601-20170613,0.926859,0.910687,0.902362,0.907881,0.902148,0.894874,0.891432,0.885742,0.879157,0.875379,0.893979,0.897652
20170625-20170707,0.910564,0.880597,0.840507,0.810327,0.793548,0.768985,0.752949,0.744466,0.740586,0.749959,0.777166,0.799249
20170719-20170731,0.948125,0.900022,0.865779,0.827202,0.802664,0.783892,0.759708,0.747308,0.727437,0.713911,0.784679,0.807605


Percentage of interferograms passes the requirement (0.683):

In [15]:
np.count_nonzero(ratio_pd['total']>thresthod)/n_ifgs

0.9183673469387755

In [16]:
np.count_nonzero(ratio_pd['mean']>thresthod)/n_ifgs

0.9591836734693877